
# **나의 해방일지**

---

# 12월 19일 <br>

Stage 3: inference<br><br>

12월 17일, 나는 모델이 내가 모은 데이터셋을 얼마나 잘 측정할 수 있는지 테스트를 해보았다. 그러나 각 숫자 별로 일부만 보여주었기 때문에,<br> 이번은 모든 데이터셋을 테스트해보았고 그 결과를 데이터프레임으로 정리하였다.


In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
import os
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%config InlineBackend.figure_format='retina'

In [4]:

torch.manual_seed(316)

In [5]:
from collections import OrderedDict

class Network(nn.Module):

  def __init__(self):
    super().__init__()
    self.feature_extraction = nn.Sequential(
            nn.Conv2d(1, 512, kernel_size=7,stride = 1, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride =1,padding= 1),
            nn.Conv2d(512, 256, kernel_size=3,stride = 1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=3,stride = 1, padding=1),
            nn.Conv2d(128, 64, kernel_size=3,stride = 1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3,stride = 1, padding=1),
            nn.Conv2d(32, 100, kernel_size=3,stride = 1, padding=1),
            nn.Conv2d(100, 200, kernel_size=3,stride = 1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5,  stride =1)
            )
    self.classifier = nn.Sequential(
            nn.Linear(22*22*200, 2048),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(2048, 10)
            )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    out1 = self.feature_extraction(x)
    out2 = torch.flatten(out1, 1)
    out3 = self.classifier(out2)
    return out3

model = Network()

In [6]:
model.load_state_dict(
    torch.load("/content/drive/MyDrive/model_1209.pth", map_location=torch.device("cpu"))
)
model.eval()

Network(
  (feature_extraction): Sequential(
    (0): Conv2d(1, 512, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=5, stride=1, padding=1, dilation=1, ceil_mode=False)
    (3): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(32, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Conv2d(100, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=96800, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [7]:
from torchsummary import summary
summary(model, (1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 512, 28, 28]          25,600
              ReLU-2          [-1, 512, 28, 28]               0
         MaxPool2d-3          [-1, 512, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]       1,179,904
              ReLU-5          [-1, 256, 26, 26]               0
            Conv2d-6          [-1, 128, 26, 26]         295,040
            Conv2d-7           [-1, 64, 26, 26]          73,792
              ReLU-8           [-1, 64, 26, 26]               0
            Conv2d-9           [-1, 32, 26, 26]          18,464
           Conv2d-10          [-1, 100, 26, 26]          28,900
           Conv2d-11          [-1, 200, 26, 26]         180,200
             ReLU-12          [-1, 200, 26, 26]               0
        MaxPool2d-13          [-1, 200, 22, 22]               0
           Linear-14                 [-

In [8]:
img_mean = [0.1307]
img_std = [0.3081]

transform = transforms.Compose(
  [
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize(img_mean, img_std),
] )

In [9]:
data_dir = "/content/drive/MyDrive/E02_data"
batch_size = 1

testset = datasets.ImageFolder(os.path.join(data_dir, "test_change"), transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

In [10]:
idx_to_class = { v:k for k, v in testset.class_to_idx.items() }
idx_to_class

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9'}

In [11]:
label = list()
model_predict = list()

In [12]:
for i in range(len(testset)):

  image, label_correct = testloader.dataset[i]
  label_correct = idx_to_class[label_correct]


  label.append(label_correct)

  with torch.no_grad():
    image_batch = image.unsqueeze(0)
    logits = model.forward(image_batch)
    probabilities = logits.softmax(dim=1)
    top_probs, top_classes = probabilities.topk(k=10, dim=1)

    label_predicted = idx_to_class[int(top_classes.squeeze()[0])]

    if label_predicted == label_correct:
      model_predict.append("correct")

    else:
      model_predict.append("incorrect")

In [13]:
df = pd.DataFrame({'label': label,
                    'model_predict': model_predict})

In [14]:
df.groupby('label')['model_predict'].value_counts()

label  model_predict
0      correct          23
1      correct          23
2      incorrect        21
       correct           2
3      incorrect        13
       correct          10
4      incorrect        15
       correct           8
5      incorrect        16
       correct           7
6      correct          13
       incorrect        10
7      incorrect        23
8      incorrect        15
       correct           8
9      incorrect        18
       correct           5
Name: model_predict, dtype: int64

In [15]:
df

,label,model_predict
0,0,correct
1,0,correct
2,0,correct
3,0,correct
4,0,correct
...,...,...
225,9,incorrect
226,9,correct
227,9,incorrect
228,9,correct


In [16]:
df['model_predict'].value_counts()

incorrect    131
correct       99
Name: model_predict, dtype: int64